In [1]:
import numpy as np
import pandas as pd
import tensorflow
import keras
from keras.layers import Convolution1D, Dense, MaxPooling1D, Flatten
from keras.models import Sequential
import itertools
%matplotlib inline



C:\local\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [35]:
x = np.load('overnight_all_compressed_new.npz')
print(x)
x_t=[]
final_array=np.array([,4])
for i in x:
    print(x[i].shape)
    temp=x[i]
    final_array = np.concatenate((final_array,temp),axis=0)
    #np.concatenate(p,x[i])
    #p =np.concatenate((p,temp),axis=0)
    #np.append(x[i])
    
#data=['seconds','mic (trachea)','abdominal movement (stretch sensor)','thorax movement (stretch sensor)']
#x = pd.DataFrame(x['arr_0'], columns = data)
#x = pd.DataFrame(x['arr_1'], columns = data)

#merged = list(itertools.chain.from_iterable(x_t))


SyntaxError: invalid syntax (<ipython-input-35-2b593dd4af4f>, line 4)

In [32]:
print(final_array.shape)

#final_array.reshape(63257524,4)
#X_test= pd.DataFrame(x['arr_1'],columns = data)

(4769867, 4)


In [ ]:
X_train = final_array['mic (trachea)']
Y_train = x['thorax movement (stretch sensor)']

In [14]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Convolution1D(filters=1,input_dim=1, padding='Same',kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, input_dim=1, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
model.fit(X_train,Y_train,epochs=1,batch_size=1000, verbose=2)
pred =model.predict(X[test])
scores = model.evaluate(pred,y[test])

In [ ]:
window_size=1
filter_length=1
nb_input_series=1
nb_outputs=2500
nb_filter=1

def oneDC_model():
    model = Sequential((
        # The first conv layer learns `nb_filter` filters (aka kernels), each of size ``(filter_length, nb_input_series)``.
        # Its output will have shape (None, window_size - filter_length + 1, nb_filter), i.e., for each position in
        # the input timeseries, the activation of each filter at that position.
        Convolution1D(nb_filter=nb_filter, filter_length=filter_length, activation='relu'),
                      #input_shape=(window_size, nb_input_series)),
        MaxPooling1D(),     # Downsample the output of convolution by 2X.
        Convolution1D(nb_filter=nb_filter, filter_length=filter_length, activation='relu'),
        MaxPooling1D(),
        Flatten(),
        Dense(nb_outputs, activation='relu'),     # For binary classification, change the activation to 'sigmoid'
    ))
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    # To perform (binary) classification instead:
    # model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
    return model

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

X = x['mic (trachea)'][:10000]
print(X.shape)
[:10000]
tscv = TimeSeriesSplit(n_splits=3)
print(tscv)  

#for train, test in tscv.split(X):
    #print(train)
    #print(y[test])
model = oneDC_model()
#x_ = X[train].reshape(1,len(train),1)

print(x_.shape)
y_=y[train].reshape(1,2500)
model.fit(X,y,epochs=1,batch_size=1000, verbose=2)

pred =model.predict(X[test])
#print('*****')
#print(pred)
scores = model.evaluate(pred,y[test])

#print(model.metrics_names)
print(scores[1])
#[1],scores[1])
print(model.metrics_names[0],scores[0])

    #print("%s %s" % (y[test], pred))
